In [25]:
import gym

# env_name = "LunarLander-v2"
# # env = gym.make(env_name)          # 导入注册器中的环境
# env = gym.make(env_name, render_mode="rgb_array")  # 使用 "human" 模式以便在屏幕上显示

import JSSEnv
env_name = "jss-v1"
# env = gym.make(env_name)          # 导入注册器中的环境
env = gym.make(env_name)  # 使用 "human" 模式以便在屏幕上显示
# env = gym.make('jss-v1', env_config={'instance_path': '/home/luanyf/Job_Shop_Scheduling_Problem_with_Reinforcement_Learning-base/dmu16'})

# print(env.action_space)

episodes = 10
for episode in range(1, episodes + 1):
    state = env.reset()           # gym风格的env开头都需要reset一下以获取起点的状态
    # print(state['action_mask'])
    done = False
    score = 0
    # env.step(env.action_space.sample() )

    while not done:
        env.render()              # 将当前的状态化成一个frame，再将该frame渲染到小窗口上
        action = env.action_space.sample()  -1   # 通过随机采样获取一个随即动作,不减1会pop from empty list
        
        n_state, reward, done, info = env.step(action)    # 将动作扔进环境中，从而实现和模拟器的交互
        # print(done)
        # n_state, reward, done, _, info = env.step(action)
        score += reward
    print("Episode : {}, Score : {}".format(episode, score))

env.close()     # 关闭窗口

/home/luanyf/miniconda3/envs/pytorch/lib/python3.9/site-packages/gym/utils/passive_env_checker.py:31: UserWarning:

WARN: A Box observation space has an unconventional shape (neither an image, nor a 1D vector). We recommend flattening the observation to have only a 1D vector or use a custom policy to properly process the data. Actual observation shape: (100, 7)

/home/luanyf/miniconda3/envs/pytorch/lib/python3.9/site-packages/gym/utils/passive_env_checker.py:174: UserWarning:

WARN: Future gym versions will require that `Env.reset` can be passed a `seed` instead of using `Env.seed` for resetting the environment random number generator.

/home/luanyf/miniconda3/envs/pytorch/lib/python3.9/site-packages/gym/utils/passive_env_checker.py:187: UserWarning:

WARN: Future gym versions will require that `Env.reset` can be passed `options` to allow the environment initialisation to be passed additional information.

/home/luanyf/miniconda3/envs/pytorch/lib/python3.9/site-packages/gym/utils/passi

Episode : 1, Score : -727.4646464646471
Episode : 2, Score : -3068.3232323232355
Episode : 3, Score : -110.33333333333343
Episode : 4, Score : -381.151515151515
Episode : 5, Score : -1601.2626262626266
Episode : 6, Score : -711.8686868686874
Episode : 7, Score : -1.636363636363618
Episode : 8, Score : -24.37373737373747
Episode : 9, Score : -1746.8787878787925
Episode : 10, Score : -2846.09090909091


In [37]:
from stable_baselines3 import DQN
from stable_baselines3.common.vec_env.dummy_vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
import gym

env_name = "jss-v1"
env = gym.make(env_name)
env = DummyVecEnv([lambda : env])
model = DQN(
    "MultiInputPolicy", 
    env=env, 
    learning_rate=5e-4,
    batch_size=128,
    buffer_size=50000,
    learning_starts=0,
    target_update_interval=250,
    policy_kwargs={"net_arch" : [256, 256]},
    verbose=1,
    tensorboard_log="./tensorboard/jss-v1/"
)

model.learn(total_timesteps=1e6)


Using cuda device
Logging to ./tensorboard/jss-v1/DQN_6


IndexError: pop from empty list

In [11]:
import tensorboard
# tensorboard -logdir .\tensorboard\LunarLander-v2\
tensorboard --logdir='/home/luanyf/Job_Shop_Scheduling_Problem_with_Reinforcement_Learning-base/tensorboard/LunarLander-v2'


SyntaxError: cannot assign to operator (1606510366.py, line 3)

In [12]:
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=10, render=False)
env.close()
mean_reward, std_reward

/home/luanyf/miniconda3/envs/pytorch/lib/python3.9/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(86.7559695132745, 88.09135911101546)

In [13]:
model.save("./model/LunarLander3.pkl")


/home/luanyf/miniconda3/envs/pytorch/lib/python3.9/site-packages/stable_baselines3/common/save_util.py:283: UserWarning: Path 'model' does not exist. Will create it.
  warnings.warn(f"Path '{path.parent}' does not exist. Will create it.")


In [58]:
env = gym.make(env_name, render_mode="rgb_array")  # 使用 "human" 模式以便在屏幕上显示
model = DQN.load("./model/LunarLander3.pkl")

state = env.reset()
# print(len(state[0]))
state = state[0]
done = False 
score = 0
while not done:
    # print(model.predict(observation=state))
    action, _ = model.predict(observation=state)
    state, reward, done, _, info = env.step(action=action)
    score += reward
    env.render()

env.close()
# score